In [1]:
# !pip install -qU llama-index llama-parse python-dotenv

In [2]:
# !pip install -qU llama-index-postprocessor-flag-embedding-reranker git+https://github.com/FlagOpen/FlagEmbedding.git

In [1]:
# !pip show torch
# Name: llama-index
# Version: 0.10.35

# Name: llama-index-postprocessor-flag-embedding-reranker
# Version: 0.1.2

Reference notebook: [LlamaParse - A Tool for Parsing Complex Documents.ipynb](https://colab.research.google.com/drive/1IVQkSGwS5kdTiKBwz85PO6vg_WaNx15c?usp=sharing#scrollTo=pLlEWwpFqFrp)

In [2]:
%load_ext autoreload
%autoreload 2

In [3]:
# from google.colab import drive
# drive.mount('/content/drive')

In [4]:
# wd_path = "/content/drive/MyDrive/Colab Notebooks/rag-notebooks"
wd_path = ".."

In [5]:
import sys
import openai
import os
import getpass
import logging

from dotenv import load_dotenv, find_dotenv

In [6]:
# os.environ["LLAMA_CLOUD_API_KEY"] = getpass.getpass("LLamaParse API Key:")

In [7]:
# os.environ["OPENAI_API_KEY"] = getpass.getpass("OpenAI API Key:")

In [8]:
# os.environ["HF_TOKEN"] = getpass.getpass("Hugging Face API Key:")

In [9]:
import nest_asyncio

nest_asyncio.apply()

In [28]:
# logging.basicConfig(stream=sys.stdout, level=logging.DEBUG)
logging.basicConfig(stream=sys.stdout, level=logging.WARNING)
logging.getLogger().addHandler(logging.StreamHandler(stream=sys.stdout))

In [33]:
from llama_index.core.callbacks.global_handlers import set_global_handler

set_global_handler("simple")
# set_global_handler("langfuse")

In [40]:
# import llama_parse
# import llama_index
# import os

# llama_parse_path = os.path.dirname(llama_parse.__file__)
# llama_index_path = os.path.dirname(llama_index.__file__)

# print(llama_parse_path)
# print(llama_index_path)

# Law questions

In [13]:
duvidas = {
    "Devedor": {
        "Contrato de Financiamento": "Quem é o Devedor? (BENEFICIÁRIA / DEVEDOR)",
        "Cédula de Crédito Bancário": "Quem é o  Devedor? (Emitente)"
    },
    "Credor": {
        "Contrato de Financiamento": "Quem é o Credor? (BNDES / FINEP)",
        "Cédula de Crédito Bancário": "Quem é o  Credor? (Credor ou Nome do banco)"
    },
    "Data de Assinatura": {
        "Contrato de Financiamento": "Qual é a data de assinatura? (Costuma estar no final do documento, após a frase `E, por estarem justos e contratados, firmam o presente em 3 (três) vias...`)",
        "Cédula de Crédito Bancário": "Qual é a data de assinatura? (Data de emissão)"
    },
    "Vencimento/Prazo": {
        "Contrato de Financiamento": "Qual é o Vencimento/Prazo? ('liquidar com a última prestação')",
        "Cédula de Crédito Bancário": "Qual é o Vencimento/Prazo? (ou Forma de pagamento, assim costuma aparecer o numero de parcelas)"
    },
    "Valor": {
        "Contrato de Financiamento": "Qual é o Valor (crédito no valor de [...])",
        "Cédula de Crédito Bancário": "Qual é o Valor? (valor do empréstimo)"
    },
    "Objeto": {
        "Contrato de Financiamento": "Qual é a FINALIDADE DO CONTRATO?",
        "Cédula de Crédito Bancário": "Qual é a Destinação do Crédito? (O que será creditado na conta corrente da Emitente?)"
    },
    "Alteração de Controle/ Reorganização": {"Contrato de Financiamento": "O que consta na clausula de Alteração de Controle/ Reorganização? (palavras chaves: 'controle direto e indireto'/ 'Vencimento Antecipado do contrato')",
                         "Cédula de Crédito Bancário": "O que consta na clausula de Alteração de Controle/ Reorganização? (palavras chaves: 'vencimento antecipado', 'inadimplemento' ou 'antecipadamente vencidas' / 'transferido o controle do capital' e/ou 'substituído')"
    },
    "Cessão": {"Contrato de Financiamento": "O que consta na clausula de cessão? (palavras chaves: 'ceder' / 'transferir')",
               "Cédula de Crédito Bancário": "O que consta na clausula de cessão? (palavras chaves: 'ceder', 'transferir', ou 'onerar a qualquer título...')"
    }
}

current_type = "Cédula de Crédito Bancário"
for search, question in duvidas.items():
    print(question)

{'Contrato de Financiamento': 'Quem é o Devedor? (BENEFICIÁRIA / DEVEDOR)', 'Cédula de Crédito Bancário': 'Quem é o  Devedor? (Emitente)'}
{'Contrato de Financiamento': 'Quem é o Credor? (BNDES / FINEP)', 'Cédula de Crédito Bancário': 'Quem é o  Credor? (Credor ou Nome do banco)'}
{'Contrato de Financiamento': 'Qual é a data de assinatura? (Costuma estar no final do documento, após a frase `E, por estarem justos e contratados, firmam o presente em 3 (três) vias...`)', 'Cédula de Crédito Bancário': 'Qual é a data de assinatura? (Data de emissão)'}
{'Contrato de Financiamento': "Qual é o Vencimento/Prazo? ('liquidar com a última prestação')", 'Cédula de Crédito Bancário': 'Qual é o Vencimento/Prazo? (ou Forma de pagamento, assim costuma aparecer o numero de parcelas)'}
{'Contrato de Financiamento': 'Qual é o Valor (crédito no valor de [...])', 'Cédula de Crédito Bancário': 'Qual é o Valor? (valor do empréstimo)'}
{'Contrato de Financiamento': 'Qual é a FINALIDADE DO CONTRATO?', 'Cédula d

In [14]:
pdf_file_path = wd_path + "/data/1.2.5.17.10.1.1 CCB 511300712 - BB.pdf"

# Load and Parse PDFs

In [15]:
from llama_parse import LlamaParse  # pip install llama-parse

# parser = LlamaParse(
#     # api_key="...",  # can also be set in your env as LLAMA_CLOUD_API_KEY
#     result_type="markdown"  # "markdown" and "text" are available
# )

parser = LlamaParse(
    result_type="markdown",
    verbose=True,
    language="pt",
    num_workers=2,
)

# sync
# documents = parser.load_data("./my_file.pdf")

# async
# docs0 = await parser.aload_data("../data/1.2.5.17.10.1.1 CCB 511300712 - BB.pdf")

# LlamaIndex Recursive Query Engine

In [42]:
from llama_index.core.settings import Settings
from llama_index.llms.openai import OpenAI
from llama_index.embeddings.openai import OpenAIEmbedding
from llama_index.core.node_parser import MarkdownElementNodeParser
from llama_index.core import VectorStoreIndex
from llama_index.core.storage.storage_context import StorageContext
from llama_index.core.indices.loading import load_index_from_storage
from llama_index.core.response.notebook_utils import display_source_node
from llama_index.core.callbacks import CallbackManager, LlamaDebugHandler

Settings.llm = OpenAI(model="gpt-3.5-turbo")
Settings.embed_model = OpenAIEmbedding(model="text-embedding-3-small")

llama_debug = LlamaDebugHandler(print_trace_on_end=True)
Settings.callback_manager = CallbackManager([llama_debug])

In [47]:
"DISPOSIÇÕES APLICÁVEIS AOS CONTRATOS DO BNDES".lower()

'disposições aplicáveis aos contratos do bndes'

In [17]:
PERSIST_DIR = wd_path + "/data/storage"

os.path.exists(PERSIST_DIR)

True

In [18]:
# check if storage already exists
if not os.path.exists(PERSIST_DIR):
    # load the documents and create the index
    # documents = SimpleDirectoryReader(llamaindex_dir + "/data").load_data()
    documents = await parser.aload_data(pdf_file_path)

    node_parser = MarkdownElementNodeParser(llm=OpenAI(model="gpt-3.5-turbo"), num_workers=8)
    nodes = node_parser.get_nodes_from_documents(documents=[documents[0]])
    base_nodes, objects = node_parser.get_nodes_and_objects(nodes)
    recursive_index = VectorStoreIndex(nodes=base_nodes+objects)
    # index = VectorStoreIndex.from_documents(documents)
    # store it for later
    # saving the embeddings to disk
    # By default, this will save the data to the directory storage, but you can change that by passing a persist_dir parameter.
    recursive_index.storage_context.persist(persist_dir=PERSIST_DIR)
else:
    # load the existing index
    storage_context = StorageContext.from_defaults(persist_dir=PERSIST_DIR)
    recursive_index = load_index_from_storage(storage_context)

In [19]:
# print(documents[0].text[:500])

# ## CEDULA DE CREDITO BANCARIO

# INTRODUÇÃO: Nr. 511.300.712

# 1. EMITENTE:

# |Razão ou denominação social:|ALLIAGE S/A INDUSTRIAS MEDICO ODONTOLOGICA|
# |---|---|
# |CNPJ:|55.979.736/0001-45|
# |Conta Corrente:|000.003.203-4|
# |Endereço:|ROD ABRAÃO ASSED S/N KM 53 MAIS 450 METROS, RECREIO ANHANGUERA|
# |Cidade:|RIBEIRÃO PRETO-SP|
# |CEP:|14.097-500|
# |E-MAIL:|fabiola.furtadoCalliage-global.com|

# 2. DADOS DA OPERAÇÃO DE CRÉDITO:

# |Valor:|R$25.000.000,00 (vinte e cinco milhões de reais)|
# |---|---|
# |Vencimento:|1

## Retriever

In [25]:
base_retriever = recursive_index.as_retriever(similarity_top_k=2)

In [34]:
retrievals = base_retriever.retrieve(
    # "Can you tell me about the key concepts for safety finetuning"
    # duvidas["Devedor"]
    # duvidas["Objeto"]["Cédula de Crédito Bancário"]
    # duvidas["Data de Assinatura"][current_type]
    duvidas["Alteração de Controle/ Reorganização"][current_type]
)

In [35]:
for n in retrievals:
    display_source_node(n, source_length=5000)

**Node ID:** acb4edff-4884-4095-b1d2-1b75b9a01c76<br>**Similarity:** 0.6369511159669077<br>**Text:** (II) O CONSELHO DE CONTROLE DE ATIVIDADES FINANCEIRAS (COAF) CONCLUIR PELA EXISTÊNCIA DE QUAISQUER DOS CRIMES DE "LAVAGEM" OU OCULTAÇÃO DE BENS, DIREITOS E/ZVALORES, POR NOS COMETIDOS POR MIM (NÓS OU no vdgsos ADMINISTRADORES/DIRIGENTES), PREVISTOS NA LEI 9_9. No SUA PRATICA 03.03.1998, OU DE FUNDADOS INDÍCIOS DE SUA PRÁTICA, Phu OU CONDENADO SER (MOS) INDICIADO(S), DENUNCIADO(S) 4. OU “CONDENADO(S) RADORES DI JUDICIALMENTE (NÓS OU NOSSOS ADMINISTRADORES" QU DIRIGENTES) IMES PELA PRÁTICAS DE QUAISQUER DESSES CRIMES: ANTECIPADO CLAUSULA ESPECIAL DE VENCIMENTO ANTECIPADOW- SEM PREJUIZO DE ANTE PADO OUTRAS CAUSAS DE VENCIMENTO ANTECIPADOY DA OBRIGACAO, STRUMENTO PREVISTA EM CLAUSULAS DESTE INSTRUMENTO OU DECORRENTES DE VENGIDA A 0 LEI, O BANCO PODERA CONSIDERAR NENSIDA A OPERACAO DE CREDITO IDACAO SE, NA E EXIGIR SUA IMEDIATA LIQUIDAÇÃO -SHKANA SUA VIGENCIA E SEM BANCO 1 EXPRESSA CONCORDANCIA DO BANCO: T)FOR TRANSFERIDO O CONTROLE DE NOSSO CAP DIRETO E/ OU INDIRETO PÉ NGESO, CAPITAL SOCIAL; II) HOUVER LARIDADE DE ALTERACAO DE TITULARIDADE DE NOSSO CAPITAL SOCIAL, STA DI ENVOLVENDO QUALQUER ACIONISTA, DIRETO OU INDIRETO, INCLUSIVE RE COM TRANSFERENCIAS ENTRE SI, COM VARIACAO SUPERIOR A 10% (DEZ TOT DO GAP POR CENTO) DO TGFAL DO GAPITAL SOCIAL; III) HOUVER ALTERACAO OCIAL DE QU DO CAPITAL SOCÉBEL, "DE 'OUALQUER DE NOSSAS CONTROLADAS QUE PERDA DE RESULTE EM PERDA, be NOSSO CONTROLE SOCIETARIO, DIRETO OU SSAS EMPRESAS INDIRETO, DESSAS. ALTERACAOEMPRESAS. COMUNICAÇÃO DENALTERAÇÃO ESTATUTÁRIA E DE DIRIGENTES - ometemo Comprometemo-nos %a comunicar ao Banco do Brasil se, durante cia a vigência da presente operação de crédito, for modificado s0 es tu nosso” estatuto” (ou, se for o caso, nosso contrato social) ou subst ui for substituído qualquer um dos nossos atuais dirigentes (ou dhi Ls ado SUSPENSAOadministradores, no caso de cooperativa). SUSPENSÃOSDE LIBERAÇÃO DE CREDITO - DECLARO-ME(AMO-NOS) CIENTE(8) DE QUE ALÉM DAS SITUAÇÕES PREVISTAS NA CLÁUSULA "VENCIMENTO EXTRAORDINÁRIO/ANTECIPADO", DESTE INSTRUMENTO, O BANCO DO BRASIL S.A. PODERÁ SUSPENDER A LIBERAÇÃO DE VALORES QUANDO DEIXAR(MOS) DE APRESENTAR AO BANCO DO BRASIL S.A. NO PRAZO POR ESTE INDICADO, A DOCUMENTAÇÃO NECESSÁRIA PARA A RENOVAÇÃO DO MEU(NOSSO) LIMITE DE CRÉDITO, BEM COMO QUANDO FOR(MOS) NEGATIVADO(S) EM QUAISQUER ÓRGÃOS DE PROTEÇÃO AO CRÉDITO OU NO CADASTRO DE EMITENTES DE CHEQUES SEM FUNDOS (CCF), TIVER (MOS) ENCERRADA(S) MINHA/NOSSA(S) CONTA (S) CORRENTE(S) EM ESTABELECIMENTO DE CRÉDITO, EM DECORRÊNCIA DE

- Continua na página 8 -—
---
 Continuação da CEDULA DE CREDITO BANCARIO nr.<br>

**Node ID:** 2423f2d8-e508-4163-be85-93d04ab7af47<br>**Similarity:** 0.5851084319265688<br>**Text:** 10.931, de 02.08.2004, e%não ZMhavérá necessidade de o cessionário ser financeira instituição Tinancéira ou entidade a ela equiparada. O ra cessionário ficarás sub-rogado em todos os direitos do cedente, podendô, inclusive cobrar os juros e demais go na f

IMPUTAÇÃO AO PAGAMENTO - Declaro-me(amos) ciente(s) que as quantias r ceb quafitiasrecebidas para crédito serão imputadas ao pagamento das ve as dás. verbas a seguir discriminadas, obrigatoriamente na seguinte ordem: multa, juros moratórios, juros remuneratórios, outros acessórios debitados, principal vencido e principal vincendo.

PREVENÇÃO E COMBATE À CORRUPÇÃO - Declaro(amos) conhecer e me (nos) comprometo (emos) a: a) respeitar o Programa de Integridade, a Política Específica de Prevenção e Combate à Lavagem de Dinheiro, ao Financiamento do Terrorismo e à Corrupção, o Código de Ética e as Normas de Conduta do BANCO DO BRASIL S.A,., especialmente as regras relacionadas a brindes, presentes, hospitalidade e evitar situações que configurem conflito de

- Continua na página 9 -
---
 Continuação da CEDULA DE CREDITO BANCARIO nr. 511.300.712, emitida nesta data por ALLIAGE S/A INDUSTRIAS MEDICO ODONTOLOGICA, em favor do Banco do Brasil S.A., no valor de R$25.000.000,00, com vencimento final em 15/08/2025.

a) respeitar Lei 12.846/2013, Decreto 8.420/2015 e qualquer legislação brasileira ou estrangeira anticorrupção, não utilizando negócio realizado, nem eventual assistência creditícia concedida ou intermediada pelo BANCO DO BRASIL S.A., como meio para cometimento de qualquer ato ilícito, inclusive contra o BANCO DO BRASIL S.A.;

b) comunicar imediatamente o BANCO DO BRASIL S.A. na ciência de situação que viole as normas previstas nas letras "a" e imen "b", e, concordar que em caso de seu descumprimento, e/ou minha (nossa) inclusão no Cadastro Nacional de Empresas Inidoneas Suspensas - CEIS, no Cadastro de Entidades Inidôneas e Suspensas - CEIS, no Cadastro de Entidades Privadas Sem Fins Lucrativos Impedidas CEPIM e/ou no Cadastro Nacional das Empresas Punidas CNEP, BANCO DO BRASIL S.A. poderá interromper ou considerar vencido antecipadamente o presente ou outros instrumentos de crédito relacionados a mim ou a meu Grupo Empresarial, sem necessidade de previa notificação judicial ou extrajudicial, ou dever de qualquer indenização.<br>

## Query engine

### Base

In [43]:
from llama_index.core.query_engine.retriever_query_engine import RetrieverQueryEngine
query_engine_base = RetrieverQueryEngine.from_args(base_retriever, verbose=True)

In [54]:
len(duvidas)

13

In [88]:
%%time

from duvidas import duvidas

# recursive_search = False
recursive_search = True
not_to_search = [
    # DONE
    "Objeto", "Data de Assinatura", "Alteração de Controle/ Reorganização", "Cessão", 
     # TODO LATER
     "Observações"]

# to_search = ["Garantias"]
# to_search = ["Cross default"]
# to_search = ["Objeto", "Cessão"]
# to_search = ["Alteração de Controle/ Reorganização"]
to_search = ["Data de Assinatura"]
# current_type = "Contrato de Financiamento"
current_type = "Cédula de Crédito Bancário"
responses = {}

original_stdout = sys.stdout
with open(wd_path + '/data/query_engine_output.txt', 'w') as f:
    # Redirect stdout to the file
    sys.stdout = f
    
    for col_name, duvida_map in duvidas.items():
        # if col_name not in not_to_search:
        if col_name in to_search:
            print("\n\n\n------------- " + col_name.upper() + "-------------\n")
            if recursive_search:
                responses[col_name] = recursive_query_engine.query(
                    duvidas[col_name][current_type]
                )
            else:
                responses[col_name] = query_engine_base.query(
                    duvidas[col_name][current_type]
                )
    # Restore the original stdout
    sys.stdout = original_stdout
# 17 seconds

CPU times: user 309 ms, sys: 354 ms, total: 662 ms
Wall time: 6.51 s


In [89]:
for col_name, response in responses.items():
    print("\n\n\n------------- " + col_name.upper() + "-------------\n")
    print(response)




------------- DATA DE ASSINATURA-------------

A data de assinatura, ou data de emissão, é 15/08/2025.


In [56]:
# ------------- ALTERAÇÃO DE CONTROLE/ REORGANIZAÇÃO-------------

# WITH RECURSIVE: A cláusula de vencimento antecipado menciona que o Banco do Brasil S.A. poderá considerar vencidas antecipadamente as operações de crédito se houver inadimplemento ou se ocorrer a transferência do controle do capital direto e/ou indireto, bem como a substituição de acionistas que resulte em perda do controle societário.

# NO RECURSIVE: A cláusula de vencimento antecipado menciona que o banco pode considerar antecipadamente vencidas as operações de crédito se houver transferência de controle direto e/ou indireto do capital, alteração na titularidade do capital social envolvendo acionistas com variação superior a 10%, ou alteração societária que resulte em perda de controle societário. Também é destacado que qualquer modificação no estatuto ou nos dirigentes deve ser comunicada ao Banco do Brasil durante a vigência da operação de crédito.


### Recursive with reranking

In [71]:
from llama_index.postprocessor.flag_embedding_reranker import FlagEmbeddingReranker

reranker = FlagEmbeddingReranker(
    top_n=5,
    model="BAAI/bge-reranker-large",
)

recursive_query_engine = recursive_index.as_query_engine(
    similarity_top_k=15,
    node_postprocessors=[reranker],
    verbose=True
)

In [23]:
# !python --version
# Python 3.10.12

response = recursive_query_engine.query(
    # duvidas["Data de Assinatura"][current_type]
    duvidas["Alteração de Controle/ Reorganização"][current_type]
    )
# MIGHT HAVE OUT OF MEMORY ISSUES

In [22]:
print(response)
# A data de assinatura da Cédula de Crédito Bancário é 18 de novembro de 2020 em Ribeirão Preto, São Paulo.

# A Alteração de Controle/Reorganização mencionada na cláusula de vencimento antecipado da cédula de crédito bancário ocorre quando há a transferência do controle direto e/ou indireto do capital, alteração de titularidade do capital social com variação superior a 10%, ou mudança na estrutura societária que resulte em perda de controle societário.

Na cláusula de Alteração de Controle/Reorganização, consta que o banco poderá considerar antecipadamente vencidas as obrigações da operação de crédito se houver transferência do controle direto e/ou indireto do capital, alteração de titularidade do capital social envolvendo qualquer acionista com variação superior a 10% do capital social, ou alteração social de qualquer controlada que resulte em perda de controle societário. Além disso, a cláusula prevê a comunicação obrigatória ao Banco do Brasil em caso de modificação do estatuto ou dos dirigentes durante a vigência da operação de crédito.
